# PyTorch：控制流和参数共享

作为动态图和权重共享的一个例子，我们实现了一个非常奇怪的模型：一个全连接的ReLU网络，在每一次前向传播时，它的隐藏层的层数为随机1到4之间的数，这样可以多次重用相同的权重来计算。

因为这个模型可以使用普通的Python流控制来实现循环，并且我们可以通过在定义转发时多次重用同一个模块来实现最内层之间的权重共享。

我们利用Mudule的子类很容易实现这个模型：

In [1]:
# 可运行代码见本文件夹中的 dynamic_net.py
import random
import torch

class DynamicNet(torch.nn.Module):
    def __init__(self, D_in, H, D_out):
        """
        在构造函数中，我们构造了三个nn.Linear实例，它们将在前向传播时被使用。
        """
        super(DynamicNet, self).__init__()
        self.input_linear = torch.nn.Linear(D_in, H)
        self.middle_linear = torch.nn.Linear(H, H)
        self.output_linear = torch.nn.Linear(H, D_out)

    def forward(self, x):
        """
        对于模型的前向传播，我们随机选择0、1、2、3，
        并重用了多次计算隐藏层的middle_linear模块。
        由于每个前向传播构建一个动态计算图，
        我们可以在定义模型的前向传播时使用常规Python控制流运算符，如循环或条件语句。
        在这里，我们还看到，在定义计算图形时多次重用同一个模块是完全安全的。
        这是Lua Torch的一大改进，因为Lua Torch中每个模块只能使用一次。
        """
        h_relu = self.input_linear(x).clamp(min=0)
        for _ in range(random.randint(0, 3)):
            h_relu = self.middle_linear(h_relu).clamp(min=0)
        y_pred = self.output_linear(h_relu)
        return y_pred


# N是批大小；D是输入维度
# H是隐藏层维度；D_out是输出维度
N, D_in, H, D_out = 64, 1000, 100, 10

# 产生输入和输出随机张量
x = torch.randn(N, D_in)
y = torch.randn(N, D_out)

# 实例化上面定义的类来构造我们的模型
model = DynamicNet(D_in, H, D_out)

# 构造我们的损失函数（loss function）和优化器（Optimizer）。
# 用平凡的随机梯度下降训练这个奇怪的模型是困难的，所以我们使用了momentum方法。
criterion = torch.nn.MSELoss(reduction='sum')
optimizer = torch.optim.SGD(model.parameters(), lr=1e-4, momentum=0.9)
for t in range(500):
    
    # 前向传播：通过向模型传入x计算预测的y。
    y_pred = model(x)

    # 计算并打印损失
    loss = criterion(y_pred, y)
    print(t, loss.item())

    # 清零梯度，反向传播，更新权重 
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

0 649.90625
1 649.3474731445312
2 645.8781127929688
3 665.173583984375
4 640.8623046875
5 614.624267578125
6 641.2725219726562
7 631.5670776367188
8 637.7459716796875
9 625.2185668945312
10 470.83551025390625
11 435.3675231933594
12 632.5811767578125
13 345.1820068359375
14 611.3906860351562
15 563.5171508789062
16 627.3644409179688
17 625.0584716796875
18 621.886962890625
19 587.0455322265625
20 612.4263305664062
21 142.03684997558594
22 598.9332275390625
23 111.41944122314453
24 94.00847625732422
25 530.2828369140625
26 564.1966552734375
27 500.838134765625
28 56.085899353027344
29 462.2825622558594
30 439.5290222167969
31 385.6937561035156
32 359.0207214355469
33 363.9302978515625
34 424.4493103027344
35 391.6949157714844
36 358.6614685058594
37 320.9956359863281
38 221.74868774414062
39 205.73878479003906
40 236.0008544921875
41 381.0203857421875
42 287.8484802246094
43 139.3165740966797
44 159.48472595214844
45 248.602294921875
46 287.087646484375
47 238.6629638671875
48 295.69122

374 98.9085922241211
375 11.944602966308594
376 1.074958324432373
377 131.6365966796875
378 101.91561889648438
379 42.543540954589844
380 7.2745256423950195
381 39.885032653808594
382 18.664888381958008
383 78.31068420410156
384 112.29149627685547
385 11.854430198669434
386 20.250028610229492
387 22.358028411865234
388 33.73491287231445
389 40.6268424987793
390 43.36037063598633
391 37.51011657714844
392 14.644497871398926
393 20.882680892944336
394 24.085248947143555
395 13.153386116027832
396 24.084169387817383
397 11.854269027709961
398 6.942853927612305
399 8.112987518310547
400 11.231542587280273
401 9.785791397094727
402 15.568102836608887
403 5.834254264831543
404 4.510124206542969
405 4.452259540557861
406 12.16606616973877
407 11.24567985534668
408 3.9041590690612793
409 9.848668098449707
410 4.092050552368164
411 2.86269211769104
412 3.7501535415649414
413 6.764093399047852
414 4.039853572845459
415 4.459043979644775
416 2.7198143005371094
417 1.673218011856079
418 7.93642854